# ZINC ID -> purchasability -> affordability

In [1]:
import csv 
import numpy as np
#import smilite
import sys
import urllib
import requests

import pandas as pd
from bs4 import BeautifulSoup
import itertools

#import urllib.parse
#from threading import Thread
#import http.client, sys
#from queue import Queue

## Curation of ZINC ID list

In [2]:
# opens csv file with zinc ids
with open('zincids.csv', 'rt') as csvfile:
    reader = csv.reader(csvfile)
    zinc_list = list(reader)

In [3]:
#remove duplicates in each list
def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list

In [4]:
print("# without duplicates removed =",len(zinc_list))
zinc_list = Remove(zinc_list)
print('# with duplicates removed =',len(zinc_list))

# without duplicates removed = 95
# with duplicates removed = 95


In [5]:
# get rid of brackets and quotations in values => zinc_list_2
zinc_list_2 = []
for zinc in zinc_list[1:]:
    string = str(zinc)
    zinc_list_2.append(string[2:-2]+" ")

# something weird going on with first value in zinc_list, fixing + appending
string0 = str(zinc_list[0])
zinc_list_2[0] = string0[8:-2]

print(len(zinc_list_2))

94


In [6]:
'''
# writing csv file with purchasable ZINC IDs
purchasable = open('zinc_purchase.txt', 'w')
for zinc in zinc_list_2:
    purchasable.write(zinc)
purchasable.close()
'''

"\n# writing csv file with purchasable ZINC IDs\npurchasable = open('zinc_purchase.txt', 'w')\nfor zinc in zinc_list_2:\n    purchasable.write(zinc)\npurchasable.close()\n"

## Fetching vendor links for each compound

In [7]:
cid = zinc_list_2[2]
print(cid)

ZINC19795634 


In [8]:
response = requests.get("http://zinc15.docking.org/substances/" + cid + "/catitems/subsets/for-sale/table.html")
response.status_code # 200 means it was downloaded successfully

200

In [9]:
xml = BeautifulSoup(response.content,"lxml")
#print(xml.prettify())

In [10]:
def vendorURLs(zinc_id_list):
    para_1 = []
    for cid in zinc_id_list:
        response = requests.get("http://zinc15.docking.org/substances/"+ cid+"/catitems/subsets/for-sale/table.html")
        xml = BeautifulSoup(response.content,"lxml")
        for i in range(0, 20):
            try:
                paragraphs = xml.find_all("td")[i]
                lines = str(paragraphs).splitlines()
                for l in lines:
                    para = str(l).splitlines()
                    for p in para:
                        if p[:12] == "<td><a href=":
                            for x in range(len(p)):
                                if p[x:x+6] == ' title':
                                    if p[12:x] not in para_1:
                                        para_1.append([p[13:x-1],cid])
            except:
                continue
    return para_1

In [11]:
compdlinks = vendorURLs(zinc_list_2)
for compL in compdlinks:
    print(compL)

['http://www.ambinter.com/reference/Amb24082520', 'ZINC28089408 ']
['https://orderbb.emolecules.com/search/#?query=261506687', 'ZINC28089408 ']
['https://orderbb.emolecules.com/search/#?query=261506687', 'ZINC28089408 ']
['http://www.otavachemicals.com/products/search-our-compounds-on-line-by-structure', 'ZINC19795634 ']
['http://www.wonder-chem.com/search.html?text=WD06U04', 'ZINC19795634 ']
['http://www.chemscene.com/goproductList/searchProductList?productObj=CS-0047', 'ZINC19795634 ']
['http://www.ambinter.com/reference/Amb17603328', 'ZINC19795634 ']
['http://www.caymanchem.com/app/template/Product.vm/catalog/10011249', 'ZINC19795634 ']
['http://www.sigmaaldrich.com/catalog/search?term=SYX00317|ALDRICH&amp;interface=All&amp;N=0&amp;mode=match%20partialmax&amp;lang=en&amp;region=US&amp;focus=product', 'ZINC06665770 ']
['http://www.sigmaaldrich.com/catalog/search?term=SYX00317&amp;interface=All&amp;N=0&amp;mode=match%20partialmax&amp;lang=en&amp;region=US&amp;focus=product', 'ZINC0666

In [12]:
file1 = open("these_are_the_mother_fucking_links.txt","a") 
for compL in compdlinks:
    for line in compL:
        if line[:6] == "mailto":
            del compL
            continue
        else:
            file1.write(line) 
            file1.write("\n")
file1.close() 

In [13]:
'''NOT: achemblblock, Ambinter, astatechinc, biochempartner, cactus.nci.nih, chemistryondemand, combi-blocks (login),
    hit2lead, labnetwork (~), molcore (no price listed), orderbb, orders.frontierssi,
    otavachemicals, ox-chem, pipharm, princetonbio, rostarglobal, specs, synquestlabs (sometimes loads, req. price),
    tocris, vitasmlab, Wonder-Chem'''
'''unknown: asis, apollo scientific, echemstore, synquest'''

'unknown: asis, apollo scientific, echemstore, synquest'

## Affordability/Vendor Filtering

In [14]:
'''
these are all good vendors:
    cayman - DONE
    chemscene - addtl click/filter - DONE
    indofinechemical - DONE
    matrixscientific - DONE
    mcule - DONE
    medchemexpress - addtl click - DONE
    molport - DONE
    targetmol - DONE
    trc-canada - DONE
    apexbt - DONE
    abovchem - DONE
    bldpharm - DONE
        keyorganicsinc - addtl click + javascript rendering
        chem-space - javascript rendering
        enaminestore - javascript rendering
        sigmaaldrich - addtl click + javascript rendering
'''

'\nthese are all good vendors:\n    cayman - DONE\n    chemscene - addtl click/filter - DONE\n    indofinechemical - DONE\n    matrixscientific - DONE\n    mcule - DONE\n    medchemexpress - addtl click - DONE\n    molport - DONE\n    targetmol - DONE\n    trc-canada - DONE\n    apexbt - DONE\n    abovchem - DONE\n    bldpharm - DONE\n        keyorganicsinc - addtl click + javascript rendering\n        chem-space - javascript rendering\n        enaminestore - javascript rendering\n        sigmaaldrich - addtl click + javascript rendering\n'

In [15]:
print(compdlinks[9][0])
for x in range(len(compdlinks)):
    if "sigma" in compdlinks[x][0]:
        print(x)

http://www.sigmaaldrich.com/catalog/search?term=SYX00317&amp;interface=All&amp;N=0&amp;mode=match%20partialmax&amp;lang=en&amp;region=US&amp;focus=product
8
9
13
14
15
16
17
39
40
60
61
65
66
71
72
75
80
81
94
95
130
131
135
136
140
141
145
146
150
151
155
156
165
166
173
174
188
189
193
194
195
196
202
203
204
205
207
208
209
210
212
213
216
217
228
229
233
234
235
236
237
249
250
251
252
270
271
272
273
274
275
276
284
285
286
287
288
299
300
309
310
311
312
318
319
327
328
333
334


In [16]:
def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list

In [17]:
def findEasyVendors(urlist):
    vendors = []
    goodvendors = ["caymanchem", "chem-space", "indofinechemical", "matrixscientific", "mcule", "molport", "apexbt", 
                  "abovchem", "bldpharm", "targetmol", "trc-canada", "chemscene", "medchemexp"]
    for row in urlist:
        for v in goodvendors:
            if v in row[0]:
                vendors.append(row)
                break
    return vendors

In [18]:
easypeesy = findEasyVendors(compdlinks)

### Abovchem

In [40]:
def abovchemPrice(urlid):
    prices = []
    size = []
    sizes = []
    units = []
    response = requests.get(urlid)
    xml = BeautifulSoup(response.content,"lxml")
    for div in xml:
        try:
            d = str(div).splitlines()
            for ind in d: 
                line = str(ind.splitlines())
                if "size: " in line:
                    for i in range(len(line)):
                        if line[i:i+6] == "size: ":
                            for j in range(len(line)):
                                if line[j:j+2] == ", ":
                                    prices.append(line[i+6:j])
                elif "className = className.replace(" in line:
                    for i in range(len(line)):
                        if line[i:i+2] == '-/':
                            for j in range(len(line)):
                                if line[j:j+2] == ", ":
                                    units.append(line[i+2:j])
                elif "count: " in line:
                    for i in range(len(line)):
                        if line[i:i+7] == "count: ":
                            for j in range(len(line)):
                                if line[j:j+2] == ", ":
                                    size.append(line[i+7:j])
        except:
            continue
                
        for x in range(len(size)):
            sizes.append(size[x]+units[x])
    aff = np.column_stack((sizes, prices))
    return aff

### MedChem Expresss

In [41]:
def medchemexpPrice(urlid):
    response = requests.get(urlid)
    xml = BeautifulSoup(response.content,"lxml")

    #if there's a space in the compound name you're going to need to replace it with "-"

    # extracting compound name from the search result
    for z in range(0,20):
        try:
            paragraphs = xml.find_all("dt")[z]
            lines = str(paragraphs).splitlines()
            for l in lines:
                if "s_pro_list_cat" in l:
                    for j in range(len(l)):
                        if l[j:j+4] == 'f="/':
                            for k in range(len(l)):
                                if l[k:k+5] == ".html":
                                    compdname = (l[j+4:k].replace(" ","-"))
        except:
            continue


    # fetching actual vendor link
    prices = []
    sizes = []
    urlid = "https://www.medchemexpress.com/" + compdname+ ".html"
    response = requests.get(urlid)
    xml = BeautifulSoup(response.content,"lxml")
    for z in range(0,20):
        paragraphs = xml.find_all("td")[z]
        lines = str(paragraphs).splitlines()
        for m in lines:
            if "USD" in m:
                if "input" not in m:
                    prices.append(m[4:].replace(" ", ""))
                else:
                    for j in range(len(m)):
                        if m[j:j+3] == ";HY":
                            line = (m[:j])
                            for k in range(len(line)):
                                if line[k] == ";":
                                    sizes.append(line[k+1:].replace(" ", ""))  
                                    
    aff = np.column_stack((sizes, prices))
    return aff

### Chemscene

In [42]:
def chemscenePrice(compdurl):
    for i in range(len(compdurl)):
        if compdurl[i:i+11] == "productObj=":
            choiceID = compdurl[i+11:]

    response = requests.get(compdurl)

    casNos = []
    vendorid = []
    xml = BeautifulSoup(response.content,"lxml")
    for div in xml:
        d = str(div).splitlines()
        for ind in d: 
            line = ind.splitlines()
            if "CAS No." in str(line):
                for i in range(len(str(line))):
                    if str(line)[i:i+2] == "</":
                        casNos.append(str(line)[23:i])
            if 'class="img-responsive"' in str(line):
                for i in range(len(str(line))):
                    if str(line)[i:i+7] == ' class=':
                        vendorid.append(str(line)[12:i-1])
    while '' in casNos:
        casNos.remove('')
    crossref = np.column_stack((casNos, vendorid))

    for i in range(len(crossref)):
        if crossref[i][1] == choiceID:
            correctcas = crossref[i][0]

    sizes = []
    prices = []
    response = requests.get("https://www.chemscene.com/" + correctcas + ".html")
    xml = BeautifulSoup(response.content,"lxml")
    for div in xml:
        d = str(div).splitlines()
        for ind in d:
            if ' prctbl-size' in str(ind):
                for i in range(len(ind)):
                    if ind[i:i+8] == 'id="size':
                        line = ind[i+11:]
                        for j in range(len(ind)):
                            if line[j:j+3] == "</s":
                                sizes.append(line[:j].replace(" ",""))
            elif 's" id="price' in ind:
                for i in range(len(ind)):
                    if ind[i:i+9] == 'id="price':
                        for j in range(len(ind)):
                            if ind[j:j+8] == "</span><":
                                 prices.append(ind[i+12:j])

    aff = np.column_stack((sizes,prices))
    return aff

### MolPort

In [43]:
def molportPrice(urlid):
    sizes = []
    prices = []
    response = requests.get(urlid)
    xml = BeautifulSoup(response.content,"lxml")
    for div in xml:
        d = str(div).splitlines()
        for ind in d:
            if '"sku"' in ind:
                for j in range(len(ind)):
                    if ind[j:j+3] == ': "':
                        for k in range(len(ind)):
                            if ind[k:k+2] == '",':
                                sizes.append(ind[j+1:k].replace(" ",""))
            if '"price"' in ind:
                for j in range(len(ind)):
                    if ind[j:j+2] == ' "':
                        for k in range(len(ind)):
                            if ind[k:k+2] == '",':
                                prices.append(ind[j+2:k])
    aff = np.column_stack((sizes, prices))
    return aff

### Apex Biotech

In [44]:
def apxbtPrice(urlid):
    sizes = []
    prices = []
    response = requests.get(urlid)
    xml = BeautifulSoup(response.content,"lxml")
    for z in range(0,20):
        paragraphs = xml.find_all("td")[z]
        lines = str(paragraphs).splitlines()
        for m in lines:
            for i in range(len(m)):
                if 'item-name">' == m[i:i+11]:
                    sizes.append(m[i+11:i+14].replace(" ",""))
                if '="price">' == m[i:i+9]:
                    for j in range(len(m)):
                        if "</span></span>" == m[j:j+14]:
                            prices.append(m[i+9:j])
    aff = np.column_stack((sizes,prices))
    return aff

### Toronto Research Chemicals - Canada

In [45]:
def trcPrice(urlid):
    prices = []
    sizes = []
    pr = []
    response = requests.get(urlid)
    xml = BeautifulSoup(response.content,"lxml")
    for z in range(0,20):
        paragraphs = xml.find_all("div")[z]
        lines = str(paragraphs).splitlines()
        for m in lines:
            if "g</td>" in m:
                if "id" not in m:
                    for x in range(len(m)):
                        if m[x:x+2] == "</":
                            sizes.append(m[23:x].replace(" ",""))
            if '<td class="optionscol">' in m:
                if '.' in m:
                    for j in range(len(m)):
                        if m[j:j+2] == "  ":
                            for i in range(len(m)):
                                if m[i] == "$":
                                    pr.append(m[i+1:j].replace(" ",""))
    prices = Remove(pr)
    aff = np.column_stack((sizes,prices))

In [67]:
for x in range(len(compdlinks)):
    if "trc" in compdlinks[x][0]:
        print(x)
        print(compdlinks[x][0])

125
https://www.trc-canada.com/product-detail/?CatNum=P167500
181
https://www.trc-canada.com/product-detail/?CatNum=B448500


In [47]:
urlid0 = compdlinks[176][0]
prices = []
sizes = []
pr = []
response = requests.get(urlid0)
xml = BeautifulSoup(response.content,"lxml")
for z in range(0,20):
    paragraphs = xml.find_all("div")[z]
    lines = str(paragraphs).splitlines()
    for m in lines:
        if "g</td>" in m:
            if "id" not in m:
                for x in range(len(m)):
                    if m[x:x+2] == "</":
                        sizes.append(m[23:x].replace(" ",""))
        if '<td class="optionscol">' in m:
            if '.' in m:
                for j in range(len(m)):
                    if m[j:j+2] == "  ":
                        for i in range(len(m)):
                            if m[i] == "$":
                                pr.append(m[i+1:j].replace(" ",""))
prices = Remove(pr)
aff = np.column_stack((sizes,prices))
for a in aff:
    print(a)

IndexError: list index out of range

### Matrix Scientific

In [52]:
def matrixPrice(urlid):
    sizes = []
    prices = []
    response = requests.get(urlid)
    xml = BeautifulSoup(response.content,"lxml")
    for z in range(0,20):
        try:
            paragraphs = xml.find_all("td")[z]
            lines = str(paragraphs).splitlines()
            for m in lines:
                if 'class="price"' in m:
                    prices.append(m[21:27].replace(",",""))
                elif "			" in m:
                    if ("<" not in m) and ("g" in m):
                        for x in range(len(m)):
                            if m[x+1] == 'g':
                                sizes.append(m[3:x+2].replace(" ",""))
        except:
            continue
        
    aff = np.column_stack((sizes,prices))
    return aff

### MCULE

In [53]:
def mculePrice(urlid):
    sizes = []
    prices = []
    response = requests.get(urlid)
    xml = BeautifulSoup(response.content,"lxml")
    for z in range(0,20):
        paragraphs = xml.find_all("td")[z]
        lines = str(paragraphs).splitlines()
        for m in lines:
            if "USD" in m:
                for x in range(len(m)):
                    if m[x:x+4] == ' USD':
                        prices.append(m[4:x])
                del m
            elif "g</td>" in m:
                if "class" not in m:
                    for x in range(len(m)):
                        if m[x:x+5] == '</td>':
                            sizes.append(m[4:x].replace(" ",""))
    aff = np.column_stack((sizes, prices))
    return aff

### Targetmol

In [54]:
def targetmolPrice(urlid):
    sizes = []
    prices = []
    response = requests.get(urlid)
    xml = BeautifulSoup(response.content,"lxml")
    for z in range(0,20):
        paragraphs = xml.find_all("td")[z]
        lines = str(paragraphs).splitlines()
        mat = lines[0].splitlines()
        for m in mat:
            if "." in str(m):
                for x in range(len(m)):
                    if m[x:x+5] == '</td>':
                        prices.append(m[4:x])
                del m
            elif " " in m:
                if "<p" not in m:
                    if "td " not in m:
                        for x in range(len(m)):
                            if m[x:x+5] == '</td>':
                                sizes.append(m[4:x].replace(" ",""))
    aff = np.column_stack((sizes, prices))
    return aff

### Indofine Chemical

In [55]:
def indofinePrice(urlid):
    sizes = []
    prices = []
    response = requests.get(urlid)
    xml = BeautifulSoup(response.content,"lxml")
    for z in range(0,20):
        paragraphs = xml.find_all("div")[z]
        lines = str(paragraphs).splitlines()
        for l in lines:
            ind = l.splitlines()
            for i in ind:
                if "gm @ $" in i[30:]:
                    sizes.append(i[28:33].replace(" ",""))
                    prices.append(i[36:42])
    aff = np.column_stack((sizes, prices))
    return aff

### Cayman Chemical

In [56]:
def caymanPrice(urlid):
    
    sizes = []
    prices = []
    response = requests.get(urlid)
    xml = BeautifulSoup(response.content,"lxml")
    for z in range(0,20):
        try:
            paragraphs = xml.find_all("td")[z]
            lines = str(paragraphs).splitlines()
            for l in lines:
                para = str(l).splitlines()
                for p in para:
                    if p[:15] == '<td class="size':
                        sizes.append(p[40:45].replace(" ",""))
                    elif p[:16] == '<td class="price':
                        prices.append(p[36:].replace(",",""))
        except:
            continue
                    
    aff = np.column_stack(((sizes, prices)))
    return aff

## Vendor Prices

In [57]:
buylist = []
for url in easypeesy:
    if "abovchem" in url[0]:
        aff = abovchemPrice(url[0])
        length = len(aff)
        rep = list(itertools.repeat(url, length))
        aff_2 = np.column_stack((aff, rep))
        buylist.append(aff_2)
    if "cayman" in url[0]:
        aff = caymanPrice(url[0])
        length = len(aff)
        rep = list(itertools.repeat(url, length))
        aff_2 = np.column_stack((aff, rep))
        buylist.append(aff_2)
    elif "indofine" in url[0]:
        aff = indofinePrice(url[0])
        length = len(aff)
        rep = list(itertools.repeat(url, length))
        aff_2 = np.column_stack((aff, rep))
        buylist.append(aff_2)
    elif "mcule" in url[0]:
        aff = mculePrice(url[0])
        length = len(aff)
        rep = list(itertools.repeat(url, length))
        aff_2 = np.column_stack((aff, rep))
        buylist.append(aff_2)
    elif "matrixscientific" in url[0]:
        aff = matrixPrice(url[0])
        length = len(aff)
        rep = list(itertools.repeat(url, length))
        aff_2 = np.column_stack((aff, rep))
        buylist.append(aff_2)
    elif "apxbt" in url[0]:
        aff = apxbtPrice(url[0])
        length = len(aff)
        rep = list(itertools.repeat(url, length))
        aff_2 = np.column_stack((aff, rep))
        buylist.append(aff_2)
    elif "molport" in url[0]:
        aff = molportPrice(url[0])
        length = len(aff)
        rep = list(itertools.repeat(url, length))
        aff_2 = np.column_stack((aff, rep))
        buylist.append(aff_2)
    elif "chemscene" in url[0]:
        aff = chemscenePrice(url[0])
        length = len(aff)
        rep = list(itertools.repeat(url, length))
        aff_2 = np.column_stack((aff, rep))
        buylist.append(aff_2);
    elif "medchemexp" in url[0]:
        aff = medchemexpPrice(url[0])
        length = len(aff)
        rep = list(itertools.repeat(url, length))
        aff_2 = np.column_stack((aff, rep))
        buylist.append(aff_2)
    elif "targetmol" in url[0]:
        aff = targetmolPrice(url[0])
        length = len(aff)
        rep = list(itertools.repeat(url, length))
        aff_2 = np.column_stack((aff, rep))
        buylist.append(aff_2)
    '''elif "trc-canada" in url[0]:
        aff = trcPrice(url[0])
        length = len(aff)
        rep = list(itertools.repeat(url, length))
        aff_2 = np.column_stack((aff, rep))
        buylist.append(aff_2)'''

In [58]:
for b in buylist:
    print(b)

[['5mg' '350.0'
  'http://www.chemscene.com/goproductList/searchProductList?productObj=CS-0047'
  'ZINC19795634 ']]
[['1mg' '36.00'
  'http://www.caymanchem.com/app/template/Product.vm/catalog/10011249'
  'ZINC19795634 ']
 ['5mg' '162.00'
  'http://www.caymanchem.com/app/template/Product.vm/catalog/10011249'
  'ZINC19795634 ']
 ['10mg' '288.00'
  'http://www.caymanchem.com/app/template/Product.vm/catalog/10011249'
  'ZINC19795634 ']
 ['50mg' '1260.00'
  'http://www.caymanchem.com/app/template/Product.vm/catalog/10011249'
  'ZINC19795634 ']]
[['1gm' '174.00' 'https://indofinechemical.com/details.aspx?Sku=08-1534'
  'ZINC06665770 ']
 ['5gm' '376.00' 'https://indofinechemical.com/details.aspx?Sku=08-1534'
  'ZINC06665770 ']
 ['1gm' '174.00' 'https://indofinechemical.com/details.aspx?Sku=08-1534'
  'ZINC06665770 ']
 ['5gm' '376.00' 'https://indofinechemical.com/details.aspx?Sku=08-1534'
  'ZINC06665770 ']]
[]
[['5mg' '60.00' 'https://www.targetmol.com/search?keyword=T1830'
  'ZINC03986669 

### Affordability Filter

In [74]:
affordable = []

desiredlimit = 100.0
desiredsize = "10mg"

for b in buylist:
    for pr in b:
        if str(pr[0]) == desiredsize:
            if float(pr[1]) <= desiredlimit:
                affordable.append(pr)
                
affordable = np.array(affordable)
affordable = affordable[np.argsort(affordable[:,1])]

### Results

In [75]:
data = {'ZINC ID': affordable[:,3],'Package Size': affordable[:,0], 'Price ($)': affordable[:,1],'Link': affordable[:,2]}
pd.set_option('display.max_colwidth', -1)
df = pd.DataFrame(data=data)
display(df)

,ZINC ID,Package Size,Price ($),Link
0,ZINC24044436,10mg,50.00,https://www.targetmol.com/search?keyword=T6277
1,ZINC01420826,10mg,50.00,https://www.targetmol.com/search?keyword=T2623
2,ZINC00006989,10mg,56.00,https://www.targetmol.com/search?keyword=T1725
3,ZINC43204048,10mg,57.00,https://www.targetmol.com/search?keyword=T3113
4,ZINC03874886,10mg,60,https://www.medchemexpress.com/search.html?q=HY-18980&amp;ft=&amp;fa=&amp;fp=
5,ZINC03815700,10mg,60,https://www.medchemexpress.com/search.html?q=HY-10256A&amp;ft=&amp;fa=&amp;fp=
6,ZINC18274812,10mg,60,https://www.medchemexpress.com/search.html?q=HY-10256A&amp;ft=&amp;fa=&amp;fp=
7,ZINC18274812,10mg,60,https://www.medchemexpress.com/search.html?q=HY-10256&amp;ft=&amp;fa=&amp;fp=
8,ZINC24044436,10mg,60,https://www.medchemexpress.com/search.html?q=HY-10320&amp;ft=&amp;fa=&amp;fp=
9,ZINC35902489,10mg,60.0,http://www.chemscene.com/goproductList/searchProductList?productObj=CS-1156
